In [0]:
%pip install yfinance pandas
%pip install requests

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import requests
import time
import pytz
from datetime import datetime
from pyspark.sql import Row

SYMBOL_CB = "ETH-USD"
API_URL = f"https://api.exchange.coinbase.com/products/{SYMBOL_CB}/ticker"
SLEEP_TIME = 5 
TABLE_NAME = "eth_raw_stream_table" 
PL_TIMEZONE = pytz.timezone('Europe/Warsaw')

print(f"Producent danych uruchomiony (Czas PL). Dane trafiają do tabeli '{TABLE_NAME}'.")

while True:
    try:
        response = requests.get(API_URL, timeout=10)
        response.raise_for_status() 
        data_json = response.json()

        #plzone
        current_time = datetime.now(PL_TIMEZONE)
        timestamp_clean = current_time.replace(tzinfo=None)
        price = float(data_json.get('price', 0.0))
        volume = float(data_json.get('volume_24h', 0.0))

        if price > 0.0:
            record = {
                "timestamp": timestamp_clean,
                "open": price, "close": price,
                "high": price, "low": price,
                "volume": volume
            }

            # Zapis do tabeli bo darmowy databrick blokuje doslownie wsztstko
            df_spark = spark.createDataFrame([Row(**record)])
            
            (df_spark.write
                     .format("delta")
                     .mode("append")
                     .saveAsTable(TABLE_NAME))

            print(f"[{timestamp_clean.strftime('%H:%M:%S')}] Zapisano: {price:.2f}")
        
    except Exception as e:
        print(f"Błąd: {e}")

    time.sleep(SLEEP_TIME)

Producent danych uruchomiony (Czas PL). Dane trafiają do tabeli 'eth_raw_stream_table'.
[21:17:25] Zapisano: 3044.90
[21:17:31] Zapisano: 3044.51
[21:17:37] Zapisano: 3044.87
[21:17:44] Zapisano: 3044.71
[21:17:50] Zapisano: 3045.38
[21:17:56] Zapisano: 3045.55
[21:18:03] Zapisano: 3045.89
[21:18:09] Zapisano: 3045.50
[21:18:16] Zapisano: 3046.09
[21:18:22] Zapisano: 3046.23
[21:18:28] Zapisano: 3046.32
[21:18:35] Zapisano: 3046.39
[21:18:42] Zapisano: 3046.40
[21:18:48] Zapisano: 3046.39
[21:18:54] Zapisano: 3046.39
[21:19:01] Zapisano: 3046.31
[21:19:07] Zapisano: 3045.73
[21:19:15] Zapisano: 3047.14
[21:19:22] Zapisano: 3053.44
[21:19:28] Zapisano: 3054.10
[21:19:35] Zapisano: 3053.72
[21:19:41] Zapisano: 3053.66
[21:19:48] Zapisano: 3053.57
[21:19:54] Zapisano: 3054.10
[21:20:00] Zapisano: 3053.79
[21:20:07] Zapisano: 3053.45
[21:20:13] Zapisano: 3053.44
[21:20:20] Zapisano: 3052.70
[21:20:26] Zapisano: 3051.99
[21:20:33] Zapisano: 3052.69
[21:20:39] Zapisano: 3052.68
[21:20:46] Za

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:139)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:139)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:136)
	at scala.collection.immutable.Range.foreach(Range.scala:192)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:721)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:441)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:441)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can